In [1]:
!nvidia-smi

Fri Jun 10 01:58:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q sentence-transformers

In [3]:
import os
import sys

import pandas as pd
import numpy as np
from torch.utils.data import DataLoader

from sentence_transformers import SentenceTransformer, models
from sentence_transformers.datasets import DenoisingAutoEncoderDataset
from sentence_transformers.losses import DenoisingAutoEncoderLoss

In [4]:
clean_legal_corpus_df = pd.read_pickle('/content/drive/MyDrive/NLP/Information_Retrieval/Legal_Text_Retrieval_Zalo2021/datasets/clean_question_answer_df.pkl')
clean_legal_corpus_df

,question_id,question,relevant_articles
0,0637bf82c8b290c7875c5bfddbf91df5,công_an xã xử_phạt lỗi mang bằng lái_xe đúng,"[{'law_id': '47/2011/tt-bca', 'article_id': '7'}]"
1,ade2b2ee4f5b869f75f0d183902382af,thực_hiện thao_tác nạp mẫu bình chứa xử_lý mẫu...,"[{'law_id': '41/2020/tt-bca', 'article_id': '1..."
2,8fa164edc7a8419cd7dc9ce66d8e695a,trình_tự đánh_giá chất_lượng hàng đối_với kiểm...,"[{'law_id': '159/2020/nđ-cp', 'article_id': '1..."
3,3aed91309b72097b34427dd28bebd98a,tử_tù chết tiêm thuốc_độc thân_nhân nhận xác h...,"[{'law_id': '53/2010/qh12', 'article_id': '60'..."
4,fe9131a8862ce1bfa7de7e2b30eeb39e,trách_nhiệm khoa_học công_nghệ quản_lý phát_tr...,"[{'law_id': '63/2020/nđ-cp', 'article_id': '20'}]"
...,...,...,...
3191,bd2f979dabdf0033a00ff488b2893e12,nội_dung kiểm_tra công_tác nghiệm_thu công_trì...,"[{'law_id': '06/2021/nđ-cp', 'article_id': '24'}]"
3192,b7347919f2bd65a2da6f9d0b1728a51c,mức phạt đối_với hành_vi chuyển mục_đích sử_dụ...,"[{'law_id': '35/2019/nđ-cp', 'article_id': '12'}]"
3193,bda416f28b8c88ce25604a26b01081cb,trích xuất thi_hành án hình_sự hiểu thế_nào,"[{'law_id': '41/2019/qh14', 'article_id': '3'}]"
3194,6088765b2e502c7d900a3f9a2f82f2b4,chạy ô_tô dầu_nhờn rơi xuống_đường phạt bao_nhiêu,"[{'law_id': '100/2019/nđ-cp', 'article_id': '2..."


In [5]:
sentences = clean_legal_corpus_df['question'].tolist()
sentences[:10]

['công_an xã xử_phạt lỗi mang bằng lái_xe đúng',
 'thực_hiện thao_tác nạp mẫu bình chứa xử_lý mẫu sơ_bộ bằng hoá_chất như_thế_nào',
 'trình_tự đánh_giá chất_lượng hàng đối_với kiểm_soát_viên quy_định như_thế_nào',
 'tử_tù chết tiêm thuốc_độc thân_nhân nhận xác hỗ_trợ mai_táng như_thế_nào',
 'trách_nhiệm khoa_học công_nghệ quản_lý phát_triển công_nghiệp an_ninh quy_định như_thế_nào',
 'đập phá biển_báo khu_vực biên_giới phạt thế_nào',
 'mục_tiêu phát_triển khu công_nghiệp hỗ_trợ quy_định như_thế_nào',
 'thừa phát tống_đạt những giấy_tờ hồ_sơ tài_liệu nào',
 'mức phạt sử_dụng trái_phép lòng_đường đô_thị hè phố rửa xe',
 'hai công_ty con chung công_ty mẹ tham_dự gói_thầu nhau']

In [6]:
train_data = DenoisingAutoEncoderDataset(sentences)
train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True, drop_last=False)

In [7]:
model = SentenceTransformer('/content/drive/MyDrive/NLP/Information_Retrieval/Legal_Text_Retrieval_Zalo2021/checkpoints/TSDAE_legal_corpus/153570')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [8]:
loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at /content/drive/MyDrive/NLP/Information_Retrieval/Legal_Text_Retrieval_Zalo2021/checkpoints/TSDAE_legal_corpus/153570/ and are newly initialized: ['encoder.layer.22.crossattention.self.value.weight', 'encoder.layer.12.crossattention.output.LayerNorm.bias', 'encoder.layer.8.crossattention.output.LayerNorm.bias', 'encoder.layer.5.crossattention.output.LayerNorm.weight', 'encoder.layer.14.crossattention.output.dense.bias', 'encoder.layer.5.crossattention.self.value.bias', 'encoder.layer.23.crossattention.output.dense.bias', 'encoder.layer.4.crossattention.self.key.bias', 'encoder.layer.2.crossattention.self.value.weight', 'encoder.layer.16.crossattention.self.key.weight', 'encoder.layer.18.crossattention.output.LayerNorm.weight', 'encoder.layer.10.crossattention.output.LayerNorm.weight', 'encoder.layer.0.crossattention.self.value.weight', 'encoder.layer.7.crossattention.output.LayerNorm.weight', 'lm_head.b

In [9]:
# !pip install -q nltk

# import nltk
# nltk.download('punkt')

In [10]:
model.fit(
    train_objectives=[(train_dataloader, loss)],
    epochs=2,
    scheduler='warmuplinear',
    optimizer_params={'lr': 1e-6},
    use_amp=True,
    show_progress_bar=True,
    checkpoint_path='/content/drive/MyDrive/NLP/Information_Retrieval/Legal_Text_Retrieval_Zalo2021/checkpoints/TSDAE_qna_corpus/',
    checkpoint_save_steps=15000,
    checkpoint_save_total_limit=2
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/799 [00:00<?, ?it/s]

Iteration:   0%|          | 0/799 [00:00<?, ?it/s]